In [1]:
import sys
sys.path.append("/home/laba/golos/whisper_fine_tuning/dataset_creation/src/")

import numpy as np
# import matplotlib.pyplot as plt
import os
import pandas as pd
from tqdm import tqdm
import librosa
import glob

from utils import load_labels
from utils_model import load_model, predict_video
from utils_analysis import calculate_wer, clean_text_before_wer

In [124]:
labels = load_labels('results/filtered_labels_train.jsonl')
# labels = load_labels('results/filtered_labels_eval.jsonl')

model_small = load_model('small')

In [114]:
model_finetune = load_model('small', model_path="experiments/small_bs_12_epoch_12_lower_lr_additional_cleaning/ckpt_epoch_best_epoch_iter_.pt")

In [125]:
prediction_finetune = predict_video(video_id = 10,
                                dataset_path = "dataset/",
                                model = model_finetune,
                                labels = labels,
                                pred_method='base')
calculate_wer(prediction_finetune)

In [116]:
# (0.19458135968521703, 0.14285714285714285)
# (0.19074436540423984, 0.14285714285714285)

# 166 - (0.26818501769276787, 0.2222222222222222)

prediction_finetune.wer.mean(), prediction_finetune.wer.median()

(0.2061442888881293, 0.12347560975609756)

In [47]:
# prediction_small = predict_video(video_id = 3,
#                                          dataset_path = "dataset/",
#                                          model = model_small,
#                                          labels = labels,
#                                          pred_method='base')

# calculate_wer(prediction_small)
# prediction_small.wer.mean(), prediction_small.wer.median()
# # (0.259469253488408, 0.23076923076923078)

In [120]:
prediction_finetune

,index,label,prediction,clean_label,clean_prediction,wer
0,0,missed audio,missed audio,missed audio,missed audio,0.000000
1,1,Пуканабабцірозривальне Британська королівська ...,кука на бабці розривальна британська королівсь...,пуканабабцірозривальне британська королівська ...,кука на бабці розривальна британська королівсь...,0.296296
2,2,Такого ніхто не очікував Ми не змогли лишитися...,такого ніхто не очікував ми не змогли лишитися...,такого ніхто не очікував ми не змогли лишитися...,такого ніхто не очікував ми не змогли лишитися...,0.156250
3,3,зроблені деякий час тому Він лисіє,зроблені деякі частому він лисіє,зроблені деякий час тому він лисіє,зроблені деякі частому він лисіє,0.500000
4,4,Але під час скандального інтерв’ю Гаррі має та...,але під час скандального інтерв’ю гаррі має та...,але під час скандального інтервю гаррі має так...,але під час скандального інтервю гаррі має так...,0.250000
...,...,...,...,...,...,...
97,97,Все так само як і з незайманістю Так тільки на...,це так само як із незайманістю ну так тільки н...,все так само як і з незайманістю так тільки на...,це так само як із незайманістю ну так тільки н...,0.391304
98,98,як на мене взагалі безпрограшне поєднання Як В...,як на мене взагалі безпрогрошне поєднання як в...,як на мене взагалі безпрограшне поєднання як в...,як на мене взагалі безпрогрошне поєднання як в...,0.028571
99,99,Ти готова трохи пробігтися сонечко Аа Діду що ...,ти готова трохи пробігтися садочко діди що ти ...,ти готова трохи пробігтися сонечко аа діду що ...,ти готова трохи пробігтися садочко діди що ти ...,0.142857
100,100,одночасно налагоджуючи стосунки із Дідом Мамою...,одночасно на лагочручі стосунки із діду мамою ...,одночасно налагоджуючи стосунки із дідом мамою...,одночасно на лагочручі стосунки із діду мамою ...,0.384615


In [24]:
prediction_small[prediction_small.wer>prediction_finetune.wer].shape

(90, 5)

In [25]:
(prediction_small.wer - prediction_finetune.wer).describe()

count    131.000000
mean       0.059494
std        0.122535
min       -0.500000
25%        0.000000
50%        0.058824
75%        0.121212
max        0.470588
Name: wer, dtype: float64

In [45]:
result = pd.concat([prediction_small, prediction_finetune.add_prefix('fine_tune_')], axis=1)

In [53]:
result[result.wer<result.fine_tune_wer].sort_values('fine_tune_wer', ascending=False).iloc[3]

label                         І поки ви в паніці кидаєте королю Таїланду в о...
prediction                     І поки ви у пані цекидаєте корлю Таїланду в о...
clean_label                   і поки ви в паніці кидаєте королю таїланду в о...
clean_prediction              і поки ви у пані цекидаєте корлю таїланду в ос...
wer                                                                     0.47619
fine_tune_label               І поки ви в паніці кидаєте королю Таїланду в о...
fine_tune_prediction          і поки ви у пані це кидаєте корлю та іланду в ...
fine_tune_clean_label         і поки ви в паніці кидаєте королю таїланду в о...
fine_tune_clean_prediction    і поки ви у пані це кидаєте корлю та іланду в ...
fine_tune_wer                                                           0.52381
Name: 36, dtype: object

In [52]:
result[result.wer<result.fine_tune_wer].sort_values('fine_tune_wer', ascending=False).iloc[3][['label', 'prediction', 'fine_tune_prediction']].values

array(['І поки ви в паніці кидаєте королю Таїланду в особисті повідомлення новий кліп Дуа Ліпи де вона співає що краще все ж таки вдома залишатись I would ve stayed at home Cause I was doing better alone Я закликаю вас не хвилюватись ',
       ' І поки ви у пані цекидаєте корлю Таїланду в особисі повідомлення новий кліп «Дуаліпи», де вона співає, що краще все ж таки дома залишатись. «А я закликаю вас не хвилюватись».',
       'і поки ви у пані це кидаєте корлю та іланду в особисі повідомлення новий кліп дуаліпи де вона співає що краще все ж таки дома залишатись я закликаю вас не хвилюватись '],
      dtype=object)

In [16]:
# df = pd.read_csv('results/labels_baseline_prediction.csv')
# df_eval = pd.read_csv('results/labels_eval_baseline_prediction.csv')

# df = pd.concat([df, df_eval])
# print(df.shape)
# df.head()

In [126]:
prediction_finetune.sort_values('wer', ascending=False).iloc[1][['label', 'prediction', 'wer']].values

array(['ніякий Як і абстрактні роздуми Зеленського про українську поліцію під час призначення нового міністра Я б дуже хотів щоб от як американським копам щоб всі наші діти довіряли українській поліції ',
       'ніякий як і абстрактні роздуми зеленського про українську поліцію під час призначення нового міністра',
       0.5333333333333333], dtype=object)

In [127]:
prediction_finetune.sort_values('wer', ascending=False).head(10)

,label,prediction,clean_label,clean_prediction,wer
130,та розповідав на пресконференціях про те як ма...,та розповідав на пресконференціях про те як ма...,та розповідав на пресконференціях про те як ма...,та розповідав на пресконференціях про те як ма...,0.620690
89,ніякий Як і абстрактні роздуми Зеленського про...,ніякий як і абстрактні роздуми зеленського про...,ніякий як і абстрактні роздуми зеленського про...,ніякий як і абстрактні роздуми зеленського про...,0.533333
142,Так і хочеться мерщій забрати до себе у владу ...,так і хочеться мерщій забрати до себе у владу ...,так і хочеться мерщій забрати до себе у владу ...,так і хочеться мерщій забрати до себе у владу ...,0.476190
80,Але зараз бачимо Зеленський значно виріс Як ко...,але зараз бачимо зеленський значно виріс як комік,але зараз бачимо зеленський значно виріс як ко...,але зараз бачимо зеленський значно виріс як комік,0.466667
32,чому глядачі потроху переходять з Кварталу на ...,чому глядачі потроху переходять з кварталу на ...,чому глядачі потроху переходять з кварталу на ...,чому глядачі потроху переходять з кварталу на ...,0.444444
84,не потребував особливих втручань нового презид...,не потребував особливих втручень нового презид...,не потребував особливих втручань нового презид...,не потребував особливих втручень нового презид...,0.421053
3,Публічний образ Зеленського тобто не такого як...,публічний образ зеленського тобто не такого як...,публічний образ зеленського тобто не такого як...,публічний образ зеленського тобто не такого як...,0.406250
99,Але Тищенко не кум Зеленського Бо наш президен...,але тищенко не кум зеленського бо наш президен...,але тищенко не кум зеленського бо наш президен...,але тищенко не кум зеленського бо наш президен...,0.400000
36,Тому тепер другий термін Зеленського лякає ще ...,тому тепер другий термін зеленського лякає ще ...,тому тепер другий термін зеленського лякає ще ...,тому тепер другий термін зеленського лякає ще ...,0.400000
128,Тоді чому міністром спорту не став друг дитинс...,тоді чому міністром спорту не став друг дитинс...,тоді чому міністром спорту не став друг дитинс...,тоді чому міністром спорту не став друг дитинс...,0.379310


In [128]:
prediction_finetune = prediction_finetune.reset_index()

In [134]:
prediction_finetune.sample()[['label', 'prediction', 'clean_label', 'clean_prediction', 'wer', 'index']].values

array([['Аж раптом з’ясувалось що олігарх Коломойський перевів 40 мільйонів доларів на офшорні рахунки 95 ого кварталу Зеленського Ви дивилися оце розслідування Слідство Інфо за результатами відкриття Пандора пейперс про офшори Бачили кіно ',
        'аж раптом з’ясувалося що олігарх коломойський перевів 40 млн доларів на офшорні рахунки 95 го кварталу зеленського ви дивилися оце розслідування слідства інфо за результатами відкриття пандора пейперс про офшори бачили кіно ',
        'аж раптом зясувалось що олігарх коломойський перевів 40 мільйонів доларів на офшорні рахунки 95 ого кварталу зеленського ви дивилися оце розслідування слідство інфо за результатами відкриття пандора пейперс про офшори бачили кіно',
        'аж раптом зясувалося що олігарх коломойський перевів 40 млн доларів на офшорні рахунки 95 го кварталу зеленського ви дивилися оце розслідування слідства інфо за результатами відкриття пандора пейперс про офшори бачили кіно',
        0.125, 102]], dtype=object)

In [ ]:
7

In [47]:
whisper_large_predictions = pd.read_csv("results/predictions.csv").dropna()
whisper_large_predictions.shape

(14651, 3)

In [48]:
calculate_wer(whisper_large_predictions)

In [49]:
path_ = "dataset/toronto_3/toronto_3_39.wav"

whisper_large_predictions[whisper_large_predictions["wav_path"] == path_][['label', 'prediction', 'wer']].values

array([[' Гадаю все буде добре Вау ти в’яжеш светр А чого такий величезний Це для моєї коханої машини Я вже зв’язав їй шарф на вихлопну трубу ',
        " Все буде добре. Моооу, ти в'яжеш свет! А чого такий величезний? Ну, це для моєї коханої машини. Я вже зв'язав її шарф на вихлопну трубу.",
        0.2]], dtype=object)

In [75]:
labels = load_labels('results/filtered_labels_train.jsonl')
